In [418]:
import pandas as pd
import math
pd.options.display.max_rows

class Forbidden:
    
    def __init__(self):
        classes = pd.read_pickle('classes.pkl')
        subjects = pd.read_pickle('subjects.pkl')
        
        self.class_sons = classes[['codigo', 'todos_filhos']]
        self.subject_sons = subjects[['codigo', 'todos_filhos']]
        
        classes['class_cod'] = classes.codigo
        subjects['subject_cod'] = subjects.codigo

        self.rules = pd.DataFrame(index=classes.class_cod, columns=subjects.subject_cod).fillna(0)
        
    def add_rule(self, cod_class, cod_subject):        
        self.rules.loc[
            self.class_sons.loc[self.class_sons['codigo'] == cod_class, 'todos_filhos'].iloc[0],
            self.subject_sons.loc[self.subject_sons['codigo'] == cod_subject, 'todos_filhos'].iloc[0]
        ] = 1
        
    def is_forbidden(self, cod_class, cod_subject):
        return self.rules.loc[cod_class, cod_subject] == 1

class Frequency:
    
    def __init__(self):
        self.subject_basic = pd.read_pickle('subject_basic.pkl')
        self.classes = pd.read_pickle('classes.pkl')
        self.subjects = pd.read_pickle('subjects.pkl')
        self.levels = {
            'class_level_0': None,
            'class_level_1': None,
            'class_level_2': None,
            'class_level_3': None,
            'class_level_4': None,
            'subject_level_0': None,
            'subject_level_1': None,
            'subject_level_2': None,
            'subject_level_3': None,
            'subject_level_4': None,
            'subject_level_5': None
        }
        self.class_level = 0
        self.subject_level = 0
        self.class_subject_freq = {}
        self.class_subject_translation = None
        self.forbidden = Forbidden()
        
    def translate_class_subject_freq(self, class_codes=[]):
        class_subject_translation = {}
        multiple_class_names = 0
        multiple_subject_names = 0

        for class_code in self.class_subject_freq:
            if len(class_codes) > 0 and class_code not in class_codes:
                continue

            class_series = self.classes.loc[self.classes['codigo'] == class_code]
            class_name = class_code if class_series.size == 0 else class_series['descricao'].iloc[0]
            class_name += ' (code: {})'.format(str(class_code))
            class_subject_translation[class_name] = {}

            for subject_code in self.class_subject_freq[class_code]:
                subject_series = self.subjects.loc[self.subjects['codigo'] == subject_code]

                subject_name = subject_code
                if subject_series.size > 0:
                    subject_name = subject_series['descricao'].iloc[0] + ' (code: {})'.format(subject_code)

                    parent_name = subject_series['cod_pai'].iloc[0]
                    if math.isnan(parent_name):
                        parent_name = None
                    else:
                        parent_name = self.subjects.loc[self.subjects['codigo'] == parent_name]['descricao'].iloc[0]

                    if parent_name:
                        subject_name = '{} >> {} (code: {})'.format(parent_name, subject_name, subject_code)

                    if self.forbidden.is_forbidden(cod_class=class_code, cod_subject=subject_code):
                        subject_name += '🚨'
    
                class_subject_translation[class_name][subject_name] = self.class_subject_freq[class_code][subject_code]

        return class_subject_translation

    def get_level_keys (levels, deepen_class=False, deepen_subject=0):
        l = ([key for key in levels.keys() if levels[key] != None])

        if 'class_level_0' not in l:
            l = l + ['class_level_0']

        if 'subject_level_0' not in l:
            l = l + ['subject_level_0']

        if deepen_class:
            class_levels = [int(c[-1:]) for c in l if c.startswith('class')]
            if len(class_levels) > 0:
                max_class_level = max(class_levels)
                if max_class_level < 4:
                    l = l + ['class_level_' + str(max_class_level+1)]

        if deepen_subject:
            subject_levels = [int(s[-1:]) for s in l if s.startswith('subject')]
            if len(subject_levels) > 0:
                max_subject_level = max(subject_levels)
                if max_subject_level < 5:
                    l = l + ['subject_level_' + str(max_subject_level+1)]

        return l
    
    
    def translate(self):
        class_subject_translation = self.translate_class_subject_freq([])

        for class_name in class_subject_translation:
            print('\n' + class_name)

            subjects_name = list(class_subject_translation[class_name].items())
            subjects_name.sort(key=lambda tup: tup[1])
            for subject_name, freq in subjects_name:
                print('\t{} - {}'.format(subject_name, freq))
                
                
    def filter_classes_subjects (self):
        df = self.subject_basic
        for level in self.levels:
            if self.levels[level] != None:
                df = df.loc[df[level] == self.levels[level]]

        return df
    
    def get_frequencies(self):
        self.class_subject_freq = self.filter_classes_subjects()
        self.class_subject_freq = self.get_class_subject_freq(self.class_subject_freq, 'cod', self.class_level, self.subject_level)

    def get_class_subject_freq(self, df, subject_col, class_level = None, subject_level = None):
        class_subject_freq = {}

        for index, row in df.iterrows():
            suit_class = row['classeProcessual']

            # If a level is given, get the class_parent in the given tree level (height)
            if class_level is not None:
                cur_classe = self.classes.loc[self.classes['codigo'] == suit_class]

                # Check if element exists in class/subject csv
                if cur_classe.size > 0:
                    parents = cur_classe.iloc[0]['cod_pais']

                    # If level is not available for that element, it is already a leaf.
                    if class_level < len(parents):
                        suit_class = parents[class_level]

            if suit_class not in class_subject_freq:
                class_subject_freq[suit_class] = {}

            subject_code = row[subject_col]

            # If a level is given, get the subject_parent in the given tree level (height)
            if subject_level is not None:
                cur_subject = self.subjects.loc[self.subjects['codigo'] == subject_code]

                # Check if element exists in class/subject csv
                if cur_subject.size > 0:
                    parents = cur_subject.iloc[0]['cod_pais']

                    # If level is not available for that element, it is already a leaf.
                    if subject_level < len(parents):
                        subject_code = parents[subject_level]

            if subject_code not in class_subject_freq[suit_class]:
                class_subject_freq[suit_class][subject_code] = 0
            class_subject_freq[suit_class][subject_code] += 1

        return class_subject_freq
        
    def open_class(self, c):
        if self.class_level > 0 and self.levels['class_level_' + str(self.class_level-1)] == None:
            self.levels['class_level_' + str(self.class_level-1)] = c
        elif self.class_level > 0 and self.levels['class_level_' + str(self.class_level-1)] != c:
            self.levels['class_level_' + str(self.class_level)] = c
        else:
            self.levels['class_level_' + str(self.class_level)] = c
    
    def open_subject(self, s):
        if self.subject_level > 0 and self.levels['subject_level_' + str(self.subject_level-1)] == None:
            self.levels['subject_level_' + str(self.subject_level-1)] = s
        else:
            self.levels['subject_level_' + str(self.subject_level)] = s
        
    def raise_subject_level(self):
        if self.subject_level < 5:
            self.subject_level+=1
    
    def raise_class_level(self):
        if self.class_level <4:
            self.class_level+=1

    def lower_subject_level(self):
        if self.subject_level > 0:
            self.subject_level-=1
    
    def lower_class_level(self):
        if self.class_level > 0:
            self.class_level-=1

In [414]:
freq = Frequency()

In [415]:
freq.forbidden.add_rule(cod_class=2, cod_subject=287)
freq.forbidden.add_rule(cod_class=2, cod_subject=1209)
freq.forbidden.add_rule(cod_class=268, cod_subject=899)
freq.forbidden.add_rule(cod_class=268, cod_subject=12734)
freq.forbidden.add_rule(cod_class=268, cod_subject=195)
freq.forbidden.add_rule(cod_class=385, cod_subject=14)

In [416]:
freq.get_frequencies()

In [417]:
freq.translate()


PROCESSO CÍVEL E DO TRABALHO (code: 2.0)
	DIREITO PENAL (code: 287)🚨 - 1
	DIREITO PROCESSUAL PENAL (code: 1209)🚨 - 1
	DIREITO INTERNACIONAL (code: 6191) - 2
	REGISTROS PÚBLICOS (code: 7724) - 8
	DIREITO PROCESSUAL CIVIL E DO TRABALHO (code: 8826) - 19
	DIREITO DO CONSUMIDOR (code: 1156.0) - 504
	DIREITO ASSISTENCIAL (code: 12734.0) - 733
	0.0 - 1363
	DIREITO CIVIL (code: 899.0) - 2055
	DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DIREITO PÚBLICO (code: 9985) - 4331
	DIREITO PREVIDENCIÁRIO (code: 195) - 5271
	DIREITO TRIBUTÁRIO (code: 14) - 12128

PROCESSO CRIMINAL (code: 268)
	DIREITO CIVIL (code: 899)🚨 - 1
	DIREITO ASSISTENCIAL (code: 12734.0)🚨 - 1
	DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DIREITO PÚBLICO (code: 9985) - 4
	DIREITO PREVIDENCIÁRIO (code: 195)🚨 - 6
	DIREITO PROCESSUAL PENAL (code: 1209) - 227
	0.0 - 979
	DIREITO PENAL (code: 287) - 6715

EXECUÇÃO PENAL E DE MEDIDAS ALTERNATIVAS (code: 385)
	DIREITO TRIBUTÁRIO (code: 14)🚨 - 1
	0.0 - 5
	DIREITO PROCESSUAL PENAL (code: 1

In [420]:
# freq = Frequency()

freq.open_class(268)
freq.open_subject(195)
freq.raise_subject_level()
# freq.raise_class_level()
freq.get_frequencies()
freq.translate()


PROCESSO CRIMINAL (code: 268)
	DIREITO PREVIDENCIÁRIO (code: 195.0)🚨 - 6


In [421]:
freq.filter_classes_subjects()

,millisInsercao,siglaTribunal,grau,numero,classeProcessual,nomeOrgao,codigoOrgao,instancia,codigoPaiNacional,cod,...,subject_level_1,subject_level_2,subject_level_3,subject_level_4,subject_level_5,class_level_0,class_level_1,class_level_2,class_level_3,class_level_4
46964,1570027601801,TRF1,G2,00368959120104019199,355,Gab. 06 - DESEMBARGADOR FEDERAL JOÃO LUIZ DE S...,13,ESP,6098.0,6098.0,...,195.0,6094.0,6094.0,6096.0,6096.0,268.0,268.0,334.0,334.0,355.0
46965,1570027601800,TRF1,G2,00368932420104019199,355,Gab. 04 - DESEMBARGADOR FEDERAL FRANCISCO NEVE...,11,ESP,6104.0,6104.0,...,195.0,6094.0,6094.0,6104.0,NaN,268.0,268.0,334.0,334.0,355.0
46968,1570027601803,TRF1,G2,00369001620104019199,355,Gab. 03 - DESEMBARGADOR FEDERAL JAMIL ROSA DE ...,10,ESP,6098.0,6098.0,...,195.0,6094.0,6094.0,6096.0,6096.0,268.0,268.0,334.0,334.0,355.0
46970,1570027601802,TRF1,G2,00368976120104019199,355,Gab. 01 - DESEMBARGADOR FEDERAL WILSON ALVES D...,7,ESP,6098.0,6098.0,...,195.0,6094.0,6094.0,6096.0,6096.0,268.0,268.0,334.0,334.0,355.0
46978,1570027601798,TRF1,G2,00368854720104019199,355,Gab. 01 - DESEMBARGADOR FEDERAL WILSON ALVES D...,7,ESP,6098.0,6098.0,...,195.0,6094.0,6094.0,6096.0,6096.0,268.0,268.0,334.0,334.0,355.0
47223,1570027604894,TRF1,G2,00396578020104019199,355,Gab. 01 - DESEMBARGADOR FEDERAL WILSON ALVES D...,7,ESP,6098.0,6098.0,...,195.0,6094.0,6094.0,6096.0,6096.0,268.0,268.0,334.0,334.0,355.0


In [402]:
freq.translate()


PROCESSO CRIMINAL (code: 268)
	DIREITO PREVIDENCIÁRIO >> Benefícios em Espécie (code: 6094) (code: 6094)🚨 - 6
